In [1]:
from ScreeningFactorData import ExpLogData, MassFractionData, ScreeningFactorData
from ScreeningFactorNetwork import ScreeningFactorNetwork
import pynucastro as pyna
import numpy as np

rng = np.random.default_rng

In [2]:
nuclei = ["h1", "he4", "c12", "o16", "n14", "ca40"]
num_nuclei = len(nuclei)
comp = pyna.Composition([pyna.Nucleus(n) for n in nuclei])

reactants = ["c12", "he4"]

temp_range = (1.e7, 1.e10)
dens_range = (1.e4, 1.e8)

size = 10**5

data_kwargs = {
    "comp": comp,
    "reactants": reactants,
    "threshold": 1.01
}

In [3]:
training = ScreeningFactorData(
    temperatures = ExpLogData.from_uniform(data_range = temp_range, size = size, rng = rng(0)),
    densities = ExpLogData.from_uniform(data_range = dens_range, size = size, rng = rng(1)),
    mass_fractions = MassFractionData.from_dirichlet(num_nuclei = num_nuclei, size = size, rng = rng(2)),
    **data_kwargs
)

testing = ScreeningFactorData(
    temperatures = ExpLogData.from_uniform(data_range = temp_range, size = size, rng = rng(3)),
    densities = ExpLogData.from_uniform(data_range = dens_range, size = size, rng = rng(4)),
    mass_fractions = MassFractionData.from_dirichlet(num_nuclei = num_nuclei, size = size, rng = rng(5)),
    **data_kwargs
)

In [4]:
training.inputs

array([[6.36961687e-01, 5.11821625e-01, 3.15002848e-02, ...,
        1.71614603e-01, 2.94273646e-01, 3.24987445e-01],
       [2.69786714e-01, 9.50463696e-01, 6.67871600e-02, ...,
        1.55136448e-01, 4.98444706e-01, 9.61838095e-02],
       [4.09735239e-02, 1.44159613e-01, 9.54844360e-02, ...,
        2.87291323e-01, 1.70207080e-01, 2.91424312e-01],
       ...,
       [7.77062361e-01, 6.64051367e-01, 1.62622741e-01, ...,
        7.43402705e-02, 9.33532695e-02, 1.27407454e-03],
       [5.79039209e-01, 4.47016407e-01, 5.45221391e-01, ...,
        2.00136996e-04, 2.14633204e-01, 3.86479497e-02],
       [5.06262615e-01, 3.91678724e-01, 4.86514601e-01, ...,
        5.02892443e-02, 3.77907906e-01, 3.78349107e-02]])

In [5]:
network = ScreeningFactorNetwork(
    training=training,
    testing=testing,
    seed=0
)

In [6]:
network.fit_model(verbose=2)

Epoch 1/20
425/425 - 3s - 8ms/step - accuracy: 0.9723 - loss: 0.0688 - val_accuracy: 0.9889 - val_loss: 0.0270
Epoch 2/20
425/425 - 2s - 5ms/step - accuracy: 0.9855 - loss: 0.0334 - val_accuracy: 0.9817 - val_loss: 0.0358
Epoch 3/20
425/425 - 2s - 5ms/step - accuracy: 0.9880 - loss: 0.0277 - val_accuracy: 0.9875 - val_loss: 0.0246
Epoch 4/20
425/425 - 2s - 5ms/step - accuracy: 0.9895 - loss: 0.0248 - val_accuracy: 0.9783 - val_loss: 0.0513
Epoch 5/20
425/425 - 2s - 5ms/step - accuracy: 0.9904 - loss: 0.0227 - val_accuracy: 0.9785 - val_loss: 0.0537


In [9]:
testing2 = ScreeningFactorData(
    temperatures = ExpLogData.from_uniform(data_range = temp_range, size = 2, rng = rng(6)),
    densities = ExpLogData.from_uniform(data_range = dens_range, size = 2, rng = rng(7)),
    mass_fractions = MassFractionData.from_static([1/6]*6, size=2),
    **data_kwargs
)

In [11]:
network.predict(testing2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[4.4287856e-14, 1.0000000e+00]], dtype=float32)